In [58]:
# Imports
import pandas as pd
import datetime

In [60]:
# Read Excel files

#  To change
folder_path = "DataSets/example-set/"

df_clients = pd.read_csv(folder_path + 'input_clients.csv')
df_instruments = pd.read_csv(folder_path + 'input_instruments.csv')
df_orders = pd.read_csv(folder_path + 'input_orders.csv')

for i in df_orders.index:
    # df_orders["Time"][i] = datetime.datetime.strptime(df_orders["Time"][i] , "%H:%M:%S").time()
    df_orders.loc[i, "Time"] = datetime.datetime.strptime(df_orders["Time"][i] , "%H:%M:%S").time()

print("Clients")
print(df_clients.to_string())

print("Instruments")
print(df_instruments.to_string())

print("Orders")
print(df_orders.to_string())

# To read ColumnName
# print(df['ColumnName'].to_string())



Clients
  ClientID   Currencies PositionCheck  Rating
0        A      USD,SGD             Y       1
1        B  USD,SGD,JPY             N       2
2        C          SGD             Y       3
3        D          USD             Y       4
4        E          SGD             N       5
Instruments
  InstrumentID Currency  LotSize
0          SIA      SGD      100
Orders
        Time OrderID Instrument  Quantity Client   Price  Side
0   09:00:01      A1        SIA      1500      A  Market   Buy
1   09:02:00      B1        SIA      4500      B    32.1  Sell
2   09:05:00      C1        SIA       100      C      32   Buy
3   09:10:00      D1        SIA       300      D  Market  Sell
4   09:29:01      B2        SIA         5      B    32.1  Sell
5   09:29:02      E1        SIA      1000      E      32  Sell
6   09:29:03      A2        SIA       800      A    31.9   Buy
7   09:30:01      C2        SIA       100      C  Market  Sell
8   09:40:00      B3        SIA       500      B    32.2  Sell
9

In [65]:
class Client:
    def __init__(self, client_id, currencies, position_check, rating):
        self.client_id = client_id
        self.currencies =currencies
        self.position_check = position_check
        self.rating = rating

    def __str__(self):
        return "{Client}: " + self.__dict__.__str__()
    

    def __repr__(self) -> str:
        return self.__str__()

clients = []

for i in df_clients.index:
    clients.append(Client(
        df_clients["ClientID"][i],
        df_clients["Currencies"][i],
        df_clients["PositionCheck"][i],
        df_clients["Rating"][i]
    ))

print(clients)

[<__main__.Client object at 0x000001A122A97450>, <__main__.Client object at 0x000001A122A9CFD0>, <__main__.Client object at 0x000001A122A9D250>, <__main__.Client object at 0x000001A122A9E290>, <__main__.Client object at 0x000001A122A9FC50>]


In [61]:
# Matching policy

# Sort by time
df_orders.sort_values(by=["Time"])
# print(df_orders.to_string())

# Seperate by time

# Open auction (09:00 - 09:30)
df_orders_open = pd.DataFrame(columns=df_orders.columns)

for i in df_orders.index:
    if (df_orders['Time'][i] < datetime.datetime.strptime("9:30:00", "%H:%M:%S").time()):
        df_orders_open = df_orders_open._append(df_orders.iloc[i], ignore_index = True)

# Continuous trading (09:30 - 16:00)
df_orders_ct = pd.DataFrame(columns=df_orders.columns)

for i in df_orders.index:
    if (datetime.datetime.strptime("9:30:00", "%H:%M:%S").time()
        <= df_orders['Time'][i]
        < datetime.datetime.strptime("16:00:00", "%H:%M:%S").time()
    ):
        df_orders_ct = df_orders_ct._append(df_orders.iloc[i], ignore_index = True)

# Close auction (16:00 - 16:10)
df_orders_close = pd.DataFrame(columns=df_orders.columns)

for i in df_orders.index:
    if (df_orders['Time'][i] >= datetime.datetime.strptime("16:00:00", "%H:%M:%S").time()):
        df_orders_close = df_orders_close._append(df_orders.iloc[i], ignore_index=True)


print("Open")
print(df_orders_open)

print("CT")
print(df_orders_ct)

print("Close")
print(df_orders_close)

Open
       Time OrderID Instrument Quantity Client   Price  Side
0  09:00:01      A1        SIA     1500      A  Market   Buy
1  09:02:00      B1        SIA     4500      B    32.1  Sell
2  09:05:00      C1        SIA      100      C      32   Buy
3  09:10:00      D1        SIA      300      D  Market  Sell
4  09:29:01      B2        SIA        5      B    32.1  Sell
5  09:29:02      E1        SIA     1000      E      32  Sell
6  09:29:03      A2        SIA      800      A    31.9   Buy
CT
       Time OrderID Instrument Quantity Client   Price  Side
0  09:30:01      C2        SIA      100      C  Market  Sell
1  09:40:00      B3        SIA      500      B    32.2  Sell
2  10:50:00      C3        SIA     4200      C    32.2   Buy
3  11:10:00      B4        SIA      100      B  Market  Sell
4  12:30:00      E2        SIA      600      E    31.9   Buy
5  13:50:00      B5        SIA     1000      B      31  Sell
Close
       Time OrderID Instrument Quantity Client Price  Side
0  16:00:01 

In [62]:
# Create order book for each instrument

order_book = {}

# order_book[<instrumentID>][<currency>]

for i in df_instruments.index:
    instrumentID = df_instruments["InstrumentID"][i]

    # Create entry in order book for instrument if not exists
    if instrumentID not in order_book:
        order_book[instrumentID] = {
            "open": {
                "Buy": pd.DataFrame(columns=df_orders.columns),
                "Sell": pd.DataFrame(columns=df_orders.columns)
            },
            "ct": {
                "Buy": pd.DataFrame(columns=df_orders.columns),
                "Sell": pd.DataFrame(columns=df_orders.columns)
            },
            "close": {
                "Buy": pd.DataFrame(columns=df_orders.columns),
                "Sell": pd.DataFrame(columns=df_orders.columns)
            },
        }
        
        # {
        #     "Buy": {
        #         "open": pd.DataFrame(columns=df_orders.columns),
        #         "ct": pd.DataFrame(columns=df_orders.columns),
        #         "close": pd.DataFrame(columns=df_orders.columns)
        #     },
        #     "Sell": {
        #         "open": pd.DataFrame(columns=df_orders.columns),
        #         "ct": pd.DataFrame(columns=df_orders.columns),
        #         "close": pd.DataFrame(columns=df_orders.columns)
        #     }
        # }
    

    # Add order to the appropriate order book
    for i in df_orders_open.index:
        if df_orders_open["Instrument"][i] == instrumentID:
            side = df_orders_open["Side"][i]
            order_book[instrumentID]["open"][side] = order_book[instrumentID]["open"][side]._append(df_orders_open.iloc[i], ignore_index=True)

    for i in df_orders_ct.index:
        if df_orders_ct["Instrument"][i] == instrumentID:
            side = df_orders_ct["Side"][i]
            order_book[instrumentID]["ct"][side] = order_book[instrumentID]["ct"][side]._append(df_orders_ct.iloc[i], ignore_index=True)

    for i in df_orders_close.index:
        if df_orders_close["Instrument"][i] == instrumentID:
            side = df_orders_close["Side"][i]
            order_book[instrumentID]["close"][side] = order_book[instrumentID]["close"][side]._append(df_orders_close.iloc[i], ignore_index=True)


for instrumentID in order_book.keys():
    print(instrumentID)
    print("Open")
    print(order_book[instrumentID]["open"]["Buy"])
    print(order_book[instrumentID]["open"]["Sell"])
    print("")

    print("CT")
    print(order_book[instrumentID]["ct"]["Buy"])
    print(order_book[instrumentID]["ct"]["Sell"])
    print("")

    print("Close")
    print(order_book[instrumentID]["close"]["Buy"])
    print(order_book[instrumentID]["close"]["Sell"])
    print("")

    print("---")

    

SIA
Open
       Time OrderID Instrument Quantity Client   Price Side
0  09:00:01      A1        SIA     1500      A  Market  Buy
1  09:05:00      C1        SIA      100      C      32  Buy
2  09:29:03      A2        SIA      800      A    31.9  Buy
       Time OrderID Instrument Quantity Client   Price  Side
0  09:02:00      B1        SIA     4500      B    32.1  Sell
1  09:10:00      D1        SIA      300      D  Market  Sell
2  09:29:01      B2        SIA        5      B    32.1  Sell
3  09:29:02      E1        SIA     1000      E      32  Sell

CT
       Time OrderID Instrument Quantity Client Price Side
0  10:50:00      C3        SIA     4200      C  32.2  Buy
1  12:30:00      E2        SIA      600      E  31.9  Buy
       Time OrderID Instrument Quantity Client   Price  Side
0  09:30:01      C2        SIA      100      C  Market  Sell
1  09:40:00      B3        SIA      500      B    32.2  Sell
2  11:10:00      B4        SIA      100      B  Market  Sell
3  13:50:00      B5     

In [57]:
# Sort the order books
def sort_df(df):
    if df.empty:
        return df
    
    # Sort by time
    df.sort_values(by=["Time"])

    # Sort by client rating
    df.sort_values(by=[""])

    print(df)

    # if df["Side"][0] == "Buy":
        
sort_df(order_book["SIA"]["open"]["Buy"])

       Time OrderID Instrument Quantity Client   Price Side
0  09:00:01      A1        SIA     1500      A  Market  Buy
1  09:05:00      C1        SIA      100      C      32  Buy
2  09:29:03      A2        SIA      800      A    31.9  Buy
